<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab 3.02: Statistical Modeling and Model Validation

> Authors: Tim Book, Matt Brems

---

## Objective
The goal of this lab is to guide you through the modeling workflow. In this lesson, you will follow all best practices when slicing your data and validating your model. The goal of this lab is not necessarily to build the best model you can, but to build and evaluate a model and interpret its results.

## Imports

In [1]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.impute import SimpleImputer

## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [2]:
# Read in the citibike data in the data folder in this repository.
citibike = pd.read_csv('./data/citibike_feb2014.csv')

## Explore the data
Use this space to familiarize yourself with the data.

If you find any issues, clean them here.

In [3]:
citibike.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,21101,Subscriber,1991,1
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,-73.989780,15456,Subscriber,1979,2
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,-73.994800,16281,Subscriber,1948,2
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,17400,Subscriber,1981,1
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,-73.989780,19341,Subscriber,1990,1


In [4]:
citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224736 entries, 0 to 224735
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             224736 non-null  int64  
 1   starttime                224736 non-null  object 
 2   stoptime                 224736 non-null  object 
 3   start station id         224736 non-null  int64  
 4   start station name       224736 non-null  object 
 5   start station latitude   224736 non-null  float64
 6   start station longitude  224736 non-null  float64
 7   end station id           224736 non-null  int64  
 8   end station name         224736 non-null  object 
 9   end station latitude     224736 non-null  float64
 10  end station longitude    224736 non-null  float64
 11  bikeid                   224736 non-null  int64  
 12  usertype                 224736 non-null  object 
 13  birth year               224736 non-null  object 
 14  gend

In [5]:
citibike.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                  object
gender                       int64
dtype: object

In [6]:
citibike.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,gender
count,224736.000000,224736.000000,224736.000000,224736.000000,224736.000000,224736.000000,224736.000000,224736.000000,224736.000000
mean,874.519810,439.203479,40.734366,-73.990386,440.741995,40.734221,-73.990521,18010.598222,1.154617
std,5486.092219,335.723861,0.019031,0.011853,341.497433,0.019048,0.011920,1987.769335,0.436592
min,60.000000,72.000000,40.680342,-74.017134,72.000000,40.680342,-74.017134,14529.000000,0.000000
25%,360.000000,305.000000,40.721854,-73.998522,305.000000,40.721816,-73.999061,16302.000000,1.000000
50%,544.000000,403.000000,40.736197,-73.990617,403.000000,40.735877,-73.990741,17975.000000,1.000000
75%,845.000000,490.000000,40.749156,-73.981918,488.000000,40.749013,-73.981948,19689.000000,1.000000
max,766108.000000,3002.000000,40.770513,-73.950048,3002.000000,40.770513,-73.950048,21542.000000,2.000000


In [7]:
citibike.isna().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

In [8]:
citibike.groupby('birth year')['birth year'].count()

birth year
1899       9
1900      68
1901      11
1907       5
1910       4
        ... 
1994    1215
1995     827
1996     334
1997     251
\N      6717
Name: birth year, Length: 78, dtype: int64

In [9]:
citibike['birth year'] = citibike['birth year'].replace('\\N', np.nan).astype(float)

In [10]:
citibike['gender'] = citibike['gender'].replace(0, np.nan)

In [11]:
citibike.isna().sum()

tripduration                  0
starttime                     0
stoptime                      0
start station id              0
start station name            0
start station latitude        0
start station longitude       0
end station id                0
end station name              0
end station latitude          0
end station longitude         0
bikeid                        0
usertype                      0
birth year                 6717
gender                     6731
dtype: int64

In [12]:
citibike['birth year'].fillna(citibike['birth year'].mean(), inplace = True)
citibike['gender'].fillna(citibike['gender'].mean(), inplace = True)

In [13]:
citibike.isna().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

In [14]:
citibike['usertype'] = np.where(citibike['usertype'] == 'subscriber',1, 0)

### Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

Null Hypothesis: Average CitiBike trip duration is not affected by gender.<br>
Alternative Hypotheses: Average CitiBike trip duration varies by gender.

In [15]:
citibike.groupby('gender')['tripduration'].mean()

gender
1.000000     814.032409
1.190266    1740.830932
2.000000     991.361074
Name: tripduration, dtype: float64

Conclulsion: Gender affects trip duration. Gender 2 has an average trip duration of 991 whereas Gender 1 has an average trip duration of 814.

### What numeric columns shouldn't be treated as numeric?

Start station id, stop station id and gender.

### Dummify the `start station id` Variable

In [16]:
# https://stackoverflow.com/questions/11587782/creating-dummy-variables-in-pandas-for-python

dummies = pd.get_dummies(citibike['start station id'], drop_first=True)

In [17]:
citibike_dummy = pd.concat([citibike, dummies], axis = 1)

In [18]:
citibike_dummy.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,2006,2008,2009,2010,2012,2017,2021,2022,2023,3002
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,...,0,0,0,0,0,0,0,0,0,0
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,...,0,0,0,0,0,0,0,0,0,0
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,...,0,0,0,0,0,0,0,0,0,0
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,...,0,0,0,0,0,0,0,0,0,0
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,...,0,0,0,0,0,0,0,0,0,0


## Feature Engineering
Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected)
- Note: you will need to clean the data a bit.

In [19]:
citibike_dummy['age'] = 2014 - citibike_dummy['birth year']

In [20]:
citibike_dummy.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,2008,2009,2010,2012,2017,2021,2022,2023,3002,age
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,...,0,0,0,0,0,0,0,0,0,23.0
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,...,0,0,0,0,0,0,0,0,0,35.0
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,...,0,0,0,0,0,0,0,0,0,66.0
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,...,0,0,0,0,0,0,0,0,0,33.0
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,...,0,0,0,0,0,0,0,0,0,24.0


## Split your data into train/test sets


Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `usertype`, `gender`, and the dummy variables you created from `start station id`. (Hint: You may find the Pandas `.drop()` method helpful here.) 

In [21]:
X = citibike_dummy.drop(citibike_dummy.iloc[:, 0:12], axis=1)

In [22]:
X = X.drop('birth year', axis = 1)

In [23]:
X.head()

,usertype,gender,79,82,83,116,119,120,127,128,...,2008,2009,2010,2012,2017,2021,2022,2023,3002,age
0,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,23.0
1,0,2.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35.0
2,0,2.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66.0
3,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33.0
4,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,24.0


In [24]:
X.shape

(224736, 331)

In [25]:
y = citibike_dummy['tripduration']

In [26]:
y.shape

(224736,)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [28]:
X_train.shape

(168552, 331)

In [29]:
X_test.shape

(56184, 331)

In [30]:
y_train.shape

(168552,)

In [31]:
y_test.shape

(56184,)

## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

In [32]:
lr = LinearRegression()

In [33]:
lr.fit(X_train, y_train)

LinearRegression()

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ to be the mean of our training `y` values.)

In [34]:
preds = lr.predict(X_test)

In [35]:
mean_squared_error(y_test, preds)

27909351.61997265

In [36]:
mean_squared_error(y_test, preds, squared = False)

5282.930211537216

In [37]:
mean_absolute_error(y_test, preds)

633.7712857199896

In [38]:
lr.score(X_train, y_train)

0.003998017251835129

In [39]:
lr.score(X_test, y_test)

-0.002471900817774708

Since the test data got a lower score than the train data, we can infer that our model is overfit.

In [40]:
mean_X = np.full_like(X, y.mean())

In [41]:
lr.score(mean_X, y)

-42911212.74470049

The model outperforms the baseline.

## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

In [42]:
X = sm.add_constant(X)

In [43]:
ols = sm.OLS(y, X).fit()

In [44]:
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           tripduration   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.264
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           2.85e-34
Time:                        16:15:44   Log-Likelihood:            -2.2535e+06
No. Observations:              224736   AIC:                         4.508e+06
Df Residuals:                  224405   BIC:                         4.511e+06
Df Model:                         330                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        641.8009    217.773      2.947      0.003     214.972    1068.630
usertype   -1.422e-11   1.55e-11     -0.920      0.358   -4.45e-11    1.61e-11
gender       179.0435     30.200      5.929      0.000     119.853     238.234
79            -3.7545    303.281     -0.012      0.990    -598.177     590.668
82           275.1695    382.029      0.720      0.471    -473.597    1023.936
83           -60.9421    379.801     -0.160      0.873    -805.342     683.458
116         -368.2684    257.902     -1.428      0.153    -873.750     137.214
119         -317.6106    756.002     -0.420      0.674   -1799.355    1164.134
120          749.0852    609.089      1.230      0.219    -444.714    1942.884
127         -295.6626    271.891     -1.087      0.277    -828.562     237.237
128         -309.9602    265.090     -1.169      0.242    -829.531     209.610
137         -258.2564    321.842     -0.802      0.422    -889.059     372.546
143         -304.2536    455.756     -0.668      0.504   -1197.523     589.016
144          -93.9880    618.070     -0.152      0.879   -1305.390    1117.414
146         -408.8105    318.005     -1.286      0.199   -1032.092     214.471
147          -94.8014    276.726     -0.343      0.732    -637.178     447.575
150           69.3106    314.927      0.220      0.826    -547.939     686.560
151         -332.2842    247.345     -1.343      0.179    -817.074     152.506
152         -353.3295    349.099     -1.012      0.311   -1037.555     330.896
153         -287.9903    270.239     -1.066      0.287    -817.651     241.670
157           82.3887    373.511      0.221      0.825    -649.683     814.460
160         -159.4250    303.149     -0.526      0.599    -753.589     434.739
161         -379.4567    263.557     -1.440      0.150    -896.022     137.109
164         -249.4846    301.097     -0.829      0.407    -839.627     340.658
167         -252.0353    275.317     -0.915      0.360    -791.649     287.578
168         -307.6134    257.975     -1.192      0.233    -813.238     198.012
173         -328.7442    275.024     -1.195      0.232    -867.784     210.295
174          277.6206    286.770      0.968      0.333    -284.441     839.683
195          292.0769    294.237      0.993      0.321    -284.620     868.774
212         -243.5861    272.372     -0.894      0.371    -777.429     290.257
216         -185.4425    679.483     -0.273      0.785   -1517.212    1146.327
217          196.0100    493.357      0.397      0.691    -770.956    1162.976
218          239.7876    663.117      0.362      0.718   -1059.904    1539.479
223         -172.3246    279.839     -0.616      0.538    -720.801     376.152
224         -307.0382    321.842     -0.954      0.340    -937.840     323.763
225            0.0410    359.931      0.000      1.000    -705.414     705.496
22

## Evaluate your model
Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

Null Hypothesis: The age variable as no significant effect when predicting trip duration <br>
Alternative Hypothesis: The age variable as no significant effect when predicting trip duration

Conclusion: The coefficient for age (4.7994) is statistically significant because its p-value of 0.000 is less than the alpha of 0.05.